# 使用 Nsight Systems 优化简单 DNN 训练程序

## 4.1 分析应用程序

对于优化工作流的第一步，我们将使用 Nsight Systems 的命令行界面（CLI）来分析我们的应用程序。由于在应用程序中执行 10 个 epoch 需要约 2 分钟，因此让我们将分析限制为第二个 epoch。第一个 epoch 意味着预热 GPU 并使程序运行，第二个 epoch 代表其余的 epochs。

我们可以通过使用程序中的 API `cudaProfilerStart` 和 `cudaProfilerStop` 来标记感兴趣的区域，如下所示。当不存在探查器时，这些API是无操作的，但它们充当探查器启动/停止集合的编程标记。

<img src=images/cudaProfilerApi.jpg width=50%>

请执行以下命令来分析应用程序的第二个 epoch。

In [ ]:
!nsys profile --trace cuda,osrt \
--capture-range cudaProfilerApi \
--output /dli/task/nsys/baseline \
--force-overwrite true \
python3 /dli/task/nsys/application/main_baseline.py

在上一个单元格运行的同时，让我们看看我们使用的命令行。

<img src=images/Commandline.jpg width=60%>

完成分析后，我们将在 Nsight Systems 的 GUI 中检查报告。执行以下单元格以生成远程桌面的 URL，您可以在其中运行 Nsight Systems 的 GUI。复制 URL 并粘贴到新的浏览器选项卡中。

In [ ]:
%%js
var url = window.location.host + '/nsight/';
element.append(url)

单击下方 `...` 以查看注释。

1. 执行上方 `%%js`单元以生成远程桌面的 URL，然后将其复制并粘贴到新的浏览器选项卡中。
2. 若提示输入密码，请输入 `nvidia`。
3. 单击桌面底部显示的应用查找工具（看起来像一个放大镜），键入 `nsight`，然后打开 `Nsight Systems`（Nsight 系统）。
4. 当提示是否允许收集遥测数据时，使用左/右箭头键选择答案，并使用 Enter 键进行选择。单击“是”将向开发团队报告遥测和崩溃，并帮助改进产品。
5. 当提示有关选项预设时，选择“CPU Rows on Top”（CPU行在顶部）模式。
5. 使用 `Ctrl+O` 键盘快捷键或单击 `File-> Open` 菜单选项打开 `/root/Desktop/reports/nsys/baseline.nsys-rep` 文件。
6. 确保打开此浏览器选项卡，因为在学习课程的过程中，您将经常返回该选项卡以检查更多报告。

## 4.2 分析配置文件

应用程序的时间线如下所示。

<img src=images/Baseline.jpg>

将鼠标悬停在 CUDA 设备行中的蓝色图表上，我们可以看到 GPU 上的 CUDA 内核覆盖率始终约为 30-40%。这意味着 GPU 在剩余的 60-70% 时间内处于空闲状态。

<img src=images/BaselineCudaKernelCoverage.jpg>

**让我们放大时间线以查看发生了什么。** 在向上或向下移动鼠标滚轮的同时按下 Ctrl 键，以放大或缩小鼠标指针周围的区域。另一种放大方法是选择要放大的区域，然后按 Shift+Z 键。

<img src=images/BaselineZoomedIn.jpg>

显然，存在一种重复的模式，其中 GPU 空闲一段时间，然后是内核和内存操作的突发。在 GPU 空闲期间，一些 CPU 线程正在执行 read/write/mmap64 操作或等待条件变量。这是由于 CPU 线程迟缓而导致 GPU 利用不足的典型例子。

为了确定 CPU 算法中的哪一步减慢了 GPU 的速度，我们需要一种方法来注释时间线以标记训练算法的阶段。为此，我们将使用[NVIDIA 工具扩展](https://nvidia.github.io/NVTX/doxygen/index.html) (NVTX) 库，它提供了一组用于注释应用程序中的事件、代码范围和资源的函数。NVTX APIs 为 NVIDIA 的工具提供了额外的信息，同时当工具未附加到应用程序时几乎不会产生任何开销。PyTorch 具有内置的 NVTX 注释，可以使用 `torch.autograd.profiler.emit_nvtx` 上下文管理器将其打开。这使得每个自动求导操作都会发出一个 NVTX 范围。

<img src=images/Built-in_NVTX.jpg width=50%>

PyTorch 中的内置 NVTX 注释通常足以分析配置文件，但是，我们还将添加自己的 NVTX 注释来标记此模块中训练算法的阶段。如果您使用的机器学习库没有内置 NVTX 注释，这将非常方便。添加 NVTX 注释的示例代码如下所示。

<img src=images/NVTX_annotations.jpg width=50%>

要查看添加 NVTX 注释的完整更改列表，请执行下面的单元格来比较 [main_baseline.py](application/main_baseline.py) 和 [main_baseline_nvtx.py](application/main_baseline_nvtx.py) 文件。

In [ ]:
!diff -y -d --color=always application/main_baseline.py application/main_baseline_nvtx.py

让我们通过执行下面给出的单元再次进行分析以获取新添加的 NVTX 注释。

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--output /dli/task/nsys/baseline_nvtx \
--force-overwrite true \
python3 /dli/task/nsys/application/main_baseline_nvtx.py

在 GUI 中打开带有 NVTX 注释的报告（baseline_nvtx.nsys-rep）并放大以找到重复的“GPU 空闲”模式，然后是“内核和内存操作的突发”。从 NVTX 注释中，我们可以看到 GPU 在训练算法的“数据加载”阶段处于空闲状态。

<img src=images/StarvationDuringDataLoading.jpg>

## 4.3 优化代码以解决 CPU 瓶颈
通过分析带有 NVTX 注释的配置文件，可以清楚地看出训练算法的数据加载步骤减慢了 GPU 的速度。让我们检查程序中使用的数据加载器 [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)。来自 [main_baseline_nvtx.py](application/main_baseline_nvtx.py) 中的代码：

<img src=images/NumberOfWorkers.jpg width=50%>

单个工作子进程正在异步加载数据。为了增加数据加载和 GPU 上的训练之间的重叠，应该增加 `num_workers` 参数。执行以下单元格以查看为调整此参数所做的代码更改：

In [ ]:
!diff -d --color=always application/main_baseline_nvtx.py application/main_opt1.py

根据目标系统上可用的 CPU 核心数量，我们可以增加“num_of_workers”来改善重叠。

## 4.4 再次进行分析以验证优化
让我们通过执行下面给出的单元再次进行分析，以验证代码更改是否解决了瓶颈。

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--output /dli/task/nsys/firstOptimization \
--force-overwrite true \
python3 /dli/task/nsys/application/main_opt1.py

在 GUI 中打开报告 (firstOptimization.nsys-rep)，并放大其中一个批次的数据加载阶段。比较优化前后的配置文件，我们注意到数据加载阶段的持续时间从约 5 毫秒减少到约 1 毫秒。这似乎并不多，但当您考虑训练算法中每个时期的数百个批次时，它就会累积起来。

<img src=images/Optimization1.jpg>

## 4.5 回顾
在本节中，您学习了如何执行以下操作：
- 使用 Nsight Systems 的命令行界面对应用程序进行分析
- 使用 cudaProfilerStart/Stop APIs 将分析限制在应用程序的某个部分
- 在应用程序中使用 NVTX API 将有关应用程序算法的更多上下文添加到 Nsight Systems 提供的时间线数据中
- 一种识别 GPU 空闲时间间隙根本原因的策略

现在我们已经解决了一个瓶颈，让我们在 [步骤 05](05_cuda_memcpy_async.ipynb) 中确定下一个性能瓶颈。